In [57]:
import numpy as np
import pandas as pd
import dask.array as da
from sklearn.linear_model import Ridge, RidgeCV

In [64]:
df = pd.DataFrame({'x': [np.array([1,2,3]), np.array([1,2,3])]})
df['x'].sum(), np.vstack(df['x'].array)

(array([2, 4, 6]),
 array([[1, 2, 3],
        [1, 2, 3]]))

In [168]:
x = da.arange(100).reshape(10, 10).rechunk(chunks=(5, 5))
y = da.arange(50).reshape(10, 5).rechunk(chunks=(5, 5))
x

dask.array<rechunk-merge, shape=(10, 10), dtype=int64, chunksize=(5, 5), chunktype=numpy.ndarray>

In [173]:
x.rechunk(chunks=(10, -1))

dask.array<rechunk-merge, shape=(10, 10), dtype=int64, chunksize=(10, 10), chunktype=numpy.ndarray>

In [144]:
def fn(x, y):
    return y - x
da.map_blocks(fn, x, y).compute()

array([[  0,   0,   0,   0,   0,  -5,  -5,  -5,  -5,  -5],
       [ -5,  -5,  -5,  -5,  -5, -10, -10, -10, -10, -10],
       [-10, -10, -10, -10, -10, -15, -15, -15, -15, -15],
       [-15, -15, -15, -15, -15, -20, -20, -20, -20, -20],
       [-20, -20, -20, -20, -20, -25, -25, -25, -25, -25],
       [-25, -25, -25, -25, -25, -30, -30, -30, -30, -30],
       [-30, -30, -30, -30, -30, -35, -35, -35, -35, -35],
       [-35, -35, -35, -35, -35, -40, -40, -40, -40, -40],
       [-40, -40, -40, -40, -40, -45, -45, -45, -45, -45],
       [-45, -45, -45, -45, -45, -50, -50, -50, -50, -50]])

### Helper Functions

-----
Finding boundaries within a contig:

In [152]:
np.diff([1, 1, 2])

array([0, 1])

In [163]:
def get_block_boundaries(x, size):
    assert x.ndim == 1
    assert size > 0
    breaks = np.argwhere(np.diff(x, prepend=x[0]))[:,0]
    breaks = np.concatenate(([0], breaks, [x.size]))
    index = np.concatenate([
        np.arange(breaks[i], breaks[i+1], size)
        for i in range(breaks.size-1)
    ])
    sizes = np.diff(index, append=x.size)
    return index, sizes
    

def test_block_boundaries():
    def check(x, size):
        idx, sizes = get_block_boundaries(x, size)
        assert sizes.sum() == x.size
        assert idx.ndim == sizes.ndim == 1
        assert idx.size == sizes.size
        chunks = []
        for i in range(idx.size):
            start, stop = idx[i], idx[i] + sizes[i]
            chunk = x[slice(start, stop)]
            assert len(chunk) <= size
            chunks.append(chunk)
        np.testing.assert_equal(np.concatenate(chunks), x)

    arrays = [
        np.array([0]),
        np.array([0, 0]),
        np.array([0, 1]),
        np.array([0, 1, 1, 1]),
        np.array([0, 1, 1, 1, 1, 10]),
        np.array([0, 1, 1, 1, 2, 2, 3, 5]),
        np.array([0, 1, 1, 2, 2, 2, 5, 5, 5, 5, 5, 8, 8, 8, 8, 10])
    ]
    for x in arrays:
        for size in [1, 2, 3]:
            check(x, size)
        check(x, x.size)

test_block_boundaries()

In [165]:
get_block_boundaries(np.array([0, 1, 1, 5, 5, 5, 8, 8, 8, 8, 10]), 2)

(array([ 0,  1,  3,  5,  6,  8, 10]), array([1, 2, 2, 1, 2, 2, 1]))

Ridge regression within blocks:

In [387]:
from sklearn.linear_model import Ridge

def ridge_regression(X, Y, a):
    """Multi-phenotype + multi-parameter ridge regression."""
    U, s, Vt = np.linalg.svd(X, full_matrices=False)
    UtY = np.dot(U.T, Y)
    V = np.expand_dims(Vt.T, 0)
    s = np.expand_dims(s, 0)
    a = np.expand_dims(a, -1)
    d = np.expand_dims(s / (s ** 2 + a), -1)
    d_UtY = d * UtY
    # returns (n_alpha, n_covariate, n_outcome)
    return np.matmul(V, d_UtY)

def test_ridge_regression():
    n, p, y = 20, 5, 3
    np.random.seed(0)
    X = np.random.normal(size=(n, p))
    B = np.random.normal(size=(p, y))
    Y = X @ B 
    alphas = np.array([0., .001, .01, .1, 1, 10, 100])
    # X, Y = X.T @ X, X.T @ Y # Regression is the same after projection -- move to separate test?
    b = ridge_regression(X, Y, alphas)
    assert b.ndim == 3
    assert b.shape == (alphas.size,) + B.shape
    # Check no regularization case
    np.testing.assert_allclose(b[0], B)
    for i, a in enumerate(alphas):
        est = Ridge(alpha=a, fit_intercept=False, normalize=False, solver='svd')
        est.fit(X, Y)
        np.testing.assert_allclose(est.coef_.T, b[i])
        
test_ridge_regression()

(20, 5) (20, 3) (5, 3)
(7, 5, 3)


In [336]:
#ridge_regression(np.random.normal(size=(3, 10)), np.random.normal(size=(3, 2)), [.01, .1])

### Implementation

In [361]:
m, n, c, y = 20, 25, 2, 3
np.random.seed(0)
X = np.random.normal(size=(n, c))
BX = np.random.normal(size=(X.shape[1], 1)) 
G = np.random.choice([0, 1, 2], size=(m, n))
BG = np.random.normal(size=(m, y))
#contigs = np.sort(np.arange(m) // 10)
contigs = np.ones(m, dtype=int)

Y = X @ BX + G.T @ BG + np.random.normal(size=(n, y), scale=.001)
X.shape, G.shape, Y.shape, contigs.shape

((25, 2), (20, 25), (25, 3), (20,))

In [362]:
contigs

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [363]:
tuple(i for i in range(3))

(0, 1, 2)

In [420]:
def _alphas(n_cols):
    # https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/ridge_model.py#L80
    heritability_vals = [0.99, 0.75, 0.50, 0.25, 0.01]
    return np.array([n_cols / h for h in heritability_vals])

def ridge_regression_cv(X, Y, alphas):
        assert alphas.ndim == 1
        assert X.ndim == 2
        assert Y.ndim == 2
        assert X.numblocks[1] == 1
        assert Y.numblocks[1] == 1
        assert X.chunks[0] == Y.chunks[0]
        # (n_covar, n_outcome)
        XtY = da.map_blocks(lambda x, y: x.T @ y, X, Y, chunks=(X.shape[1], Y.shape[1]))
        # (n_block, n_covar, n_outcome)
        XtY = da.stack([XtY.blocks[i] for i in range(XtY.numblocks[0])])
        # (n_block*n_covar, n_covar)
        XtX = da.map_blocks(lambda x: x.T @ x, X, chunks=(X.shape[1],)*2)
        # (n_block, n_covar, n_covar)
        XtX = da.stack([XtX.blocks[i] for i in range(XtX.numblocks[0])])
        # Subtract from sum across block dim
        XtX = XtX.sum(axis=0) - XtX
        # Regress for all outcomes/alphas
        B = da.map_blocks(
            # returns (n_alpha, n_covariate, n_outcome)
            lambda x, y: ridge_regression(x[0], y[0], a=alphas),
            XtX, XtY, chunks=(alphas.size, X.shape[1], Y.shape[1])
        )
        # Compute predictions for all outcomes/alphas
        YP = da.map_blocks(
            lambda x, b: x @ b, X, B, 
            chunks=(alphas.size, X.chunks[0], Y.shape[1])
        )
        return X, B, YP
    
def _level_0(G, X, Y, contigs):
    assert G.ndim == 2
    assert X.ndim == 2
    assert Y.ndim == 2
    assert len(set(map(len, [G, X, Y]))) == 1
    n_sample, n_variant = G.shape
    n_covar = X.shape[1]
    alphas = _alphas(n_variant)
    
    # Should G be scaled first?
    # https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/functions.py#L107
    arrs = []
    for vi in range(G.numblocks[1]):
        # (n_samples, n_variant)
        GB = G.blocks[:,vi]
        # (n_samples, n_variant + n_covar)
        XGB = da.concatenate((X, GB), axis=1)
        XGB = XGB.rechunk(chunks=(None, -1))
        arrs.append(ridge_regression_cv(XGB, Y, alphas))
    return arrs
    

# def _level_1(YB, Y, bc):
#     assert YB.ndim == 4
#     assert Y.ndim == 2
#     assert bc.ndim == 1
#     n_outcome, n_block, n_alpha, n_sample = YB.shape
#     assert bc.size == n_block
#     # https://github.com/projectglow/glow/blob/f3edf5bb8fe9c2d2e1a374d4402032ba5ce08e29/python/glow/wgr/linear_model/ridge_model.py#L80
#     alphas = _alphas(n_alpha * n_block)
#     for i in range(n_outcome):
#         for contig in np.unique(bc):
#             train_bidx = np.argwhere(bc != contig)[:,0]
#             pred_bidx = np.argwhere(bc == contig)[:,0]
#             # Extract (n_block_subset, n_alpha, n_sample)
#             X_train = YB[i, train_bidx]
#             X_pred = YB[i, pred_bidx]
#             Y_train = Y[:, i]
#             print(X_train.shape, X_pred.shape, Y_train.shape)
            
#             RidgeCV(alphas=alphas, fit_intercept=False, normalize=False, scoring='r2')
            

def regenie(G, X, Y, contigs, variant_block_size, sample_block_size, orthogonalize=False, add_intercept=True):
    assert len(set(map(len, [G, X, Y]))) == 1
    n_sample, n_variant = Y.shape[0], G.shape[1]
    
    G, X, Y = da.asarray(G), da.asarray(X), da.asarray(Y)
    if X is None:
        X = da.zeros(shape=(n_sample, 0), dtype=G.dtype)
    if add_intercept:
        X = da.concatenate((da.ones(shape=(n_sample, 1), dtype=X.dtype), X), axis=1)
    if orthogonalize:
        G = G - X @ da.linalg.lstsq(X, G, rcond=None)[0]
        Y = Y - X @ da.linalg.lstsq(X, Y, rcond=None)[0]
        G = G / G.std(axis=0)
        Y = Y / Y.std(axis=0)
        X = da.zeros(shape=(n_sample, 0), dtype=G.dtype)
        
    variant_block_index, variant_block_size = get_block_boundaries(contigs, variant_block_size)
    G = G.rechunk(chunks=(sample_block_size, tuple(variant_block_size)))
    X = X.rechunk(chunks=(sample_block_size, -1))
    Y = Y.rechunk(chunks=(sample_block_size, -1))
  
    R = _level_0(G, X, Y, contigs)
    return G, X, Y, R
#     YB, bc = _level_0(G, X, Y, contigs, block_size=block_size)
#     assert len(np.unique(bc)) == len(np.unique(contigs))
#     _level_1(YB, Y, bc)
#     print(YB.shape)

# TODO: mean scale Y
res = regenie(G.T, X, Y, contigs, variant_block_size=8, sample_block_size=5)
res[-1]

[(dask.array<rechunk-merge, shape=(25, 11), dtype=float64, chunksize=(5, 11), chunktype=numpy.ndarray>,
  dask.array<lambda, shape=(25, 11, 3), dtype=float64, chunksize=(5, 11, 3), chunktype=numpy.ndarray>,
  dask.array<lambda, shape=(25, 25, 3), dtype=float64, chunksize=(5, 5, 3), chunktype=numpy.ndarray>),
 (dask.array<rechunk-merge, shape=(25, 11), dtype=float64, chunksize=(5, 11), chunktype=numpy.ndarray>,
  dask.array<lambda, shape=(25, 11, 3), dtype=float64, chunksize=(5, 11, 3), chunktype=numpy.ndarray>,
  dask.array<lambda, shape=(25, 25, 3), dtype=float64, chunksize=(5, 5, 3), chunktype=numpy.ndarray>),
 (dask.array<rechunk-merge, shape=(25, 7), dtype=float64, chunksize=(5, 7), chunktype=numpy.ndarray>,
  dask.array<lambda, shape=(25, 7, 3), dtype=float64, chunksize=(5, 7, 3), chunktype=numpy.ndarray>,
  dask.array<lambda, shape=(25, 25, 3), dtype=float64, chunksize=(5, 5, 3), chunktype=numpy.ndarray>)]

In [421]:
a = res[-1][0][0]
a

dask.array<rechunk-merge, shape=(25, 11), dtype=float64, chunksize=(5, 11), chunktype=numpy.ndarray>

In [415]:
res[-1][0][-3]

dask.array<rechunk-merge, shape=(25, 11), dtype=float64, chunksize=(5, 11), chunktype=numpy.ndarray>

In [416]:
res[-1][0][-2]

dask.array<lambda, shape=(20, 11, 3), dtype=float64, chunksize=(4, 11, 3), chunktype=numpy.ndarray>

In [417]:
res[-1][0][-1]

dask.array<fn, shape=(20, 25, 3), dtype=float64, chunksize=(4, 5, 3), chunktype=numpy.ndarray>

In [418]:
res[-1][0][-1].compute_chunk_sizes().chunks

(5, 11)(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11)(5, 11) (4, 11, 3)
 (5, 11) (4, 11, 3)
(4, 11, 3)
 (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (5, 11)(4, 11, 3)
 (4, 11, 3)


((4, 4, 4, 4, 4), (5, 5, 5, 5, 5), (3,))

In [412]:
res[-1][0][-1].compute(scheduler='single-threaded').shape

(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)
(5, 11) (4, 11, 3)


(20, 25, 3)